I read about the impending changes to the strike zone in Major League Baseball and the inevitable compaints that it would kill the game, umps miscall pitches regardless, stop ruining our pasttime/they took our jobs, etc.  I decided to look at the data to see how good umps are in general and what the proposed changes would do.

## Data

I'm using a subset of the pitchF/X data.  I downloaded a copy of it in sql format, tried loading it into sqlite without luck sine it's a mySQL .sql file.  Then I tried mySQL and it compained about permissions so I had to figure that out.  Then I tried importing it to HDFS using sqoop and had several issues including figuring out the connection string, downloading a mySQL driver/connector, allowing remote access, setting passwords (and then ignoring them? wtf), and making a non-root access account.  It eventually worked.  Eventually.  So now I have the relevant tables loaded into HDFS and I'll analyze the data with Spark.

Incidentally, it's only about 2GB of data uncompressed, so it's definitely possible with a local database, but I'd like to practice with Spark and setting up an environment plus later the iterative ML techniques we'll use aren't terribly helped by having the data tied up in a database.

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
print sc

In [3]:
!hdfs dfs -ls /user/hadoop

Found 7 items
drwxr-xr-x   - hadoop hadoop          0 2016-05-22 21:48 /user/hadoop/.sparkStaging
drwxr-xr-x   - hadoop hadoop          0 2016-05-22 16:11 /user/hadoop/UMPIRETMP2
drwxr-xr-x   - hadoop hadoop          0 2016-05-22 16:11 /user/hadoop/atbats
drwxr-xr-x   - hadoop hadoop          0 2016-05-22 16:12 /user/hadoop/averages
drwxr-xr-x   - hadoop hadoop          0 2016-05-22 16:39 /user/hadoop/games
drwxr-xr-x   - hadoop hadoop          0 2016-05-22 16:40 /user/hadoop/pitches
drwxr-xr-x   - hadoop hadoop          0 2016-05-22 16:28 /user/hadoop/umpires


In [4]:
!hdfs dfs -ls /user/hadoop/pitches

Found 5 items
-rw-r--r--   2 hadoop hadoop          0 2016-05-22 16:40 /user/hadoop/pitches/_SUCCESS
-rw-r--r--   2 hadoop hadoop  372256887 2016-05-22 16:40 /user/hadoop/pitches/part-m-00000
-rw-r--r--   2 hadoop hadoop  309153780 2016-05-22 16:40 /user/hadoop/pitches/part-m-00001
-rw-r--r--   2 hadoop hadoop  374243302 2016-05-22 16:40 /user/hadoop/pitches/part-m-00002
-rw-r--r--   2 hadoop hadoop  374633764 2016-05-22 16:40 /user/hadoop/pitches/part-m-00003


In [3]:
pitchRDD = sc.textFile("/user/hadoop/pitches/*").map(lambda x: (x.split(',')[0], x.split(',')[1:]))

In [4]:
pitchRDD.take(1)

[(u'13',
  [u'4',
   u'Ball',
   u'B',
   u'360',
   u'72.1',
   u'177.01',
   u'84.9',
   u'78.8',
   u'3.386',
   u'1.698',
   u'-6.464',
   u'4.097',
   u'0.825',
   u'1.2',
   u'-0.857',
   u'55.0',
   u'5.841',
   u'6.012',
   u'-124.291',
   u'-4.571',
   u'-10.084',
   u'21.885',
   u'-25.708',
   u'26.3',
   u'19.0',
   u'8.8',
   u'1',
   u'2',
   u'null',
   u'421684',
   u'null',
   u'070410_181850',
   u'CH',
   u'1.03614339640929',
   u'null'])]

We split the line so the pitch_id is the key and the value is a list.  Here's what each entry in the list is:

| index | field           |
|-------|-----------------|
| 0     | ab_id           |
| 1     | des             |
| 2     | type            |
| 3     | id              |
| 4     | x               |
| 5     | y               |
| 6     | start_speed     |
| 7     | end_speed       |
| 8     | sz_top          |
| 9     | sz_bot          |
| 10    | pfx_x           |
| 11    | pfx_z           |
| 12    | px              |
| 13    | pz              |
| 14    | x0              |
| 15    | y0              |
| 16    | z0              |
| 17    | vx0             |
| 18    | vy0             |
| 19    | vz0             |
| 20    | ax              |
| 21    | ay              |
| 22    | az              |
| 23    | break_y         |
| 24    | break_angle     |
| 25    | break_length    |
| 26    | ball            |
| 27    | strike          |
| 28    | on_1b           |
| 29    | on_2b           |
| 30    | on_3b           |
| 31    | sv_id           |
| 32    | pitch_type      |
| 33    | type_confidence |
| 34    | my_pitch_type   |
| 35    | my_pitch_type   |



To check overall accuracy, really we just need the sz_top, sz_bottom, px, pz, and des.  Later we'll identify other variables but for now we're going to check out the pitch calls in aggregate.

In [7]:
pitchCounts = pitchRDD.map(lambda x: (x[1][1], 1)).reduceByKey(lambda x, y: x + y).collect()

In [8]:
pitchCounts

[(u'', 86),
 (u'Hit By Pitch', 15425),
 (u'Foul (Runner Going)', 25171),
 (u'Automatic Ball', 56),
 (u'In play', 1285668),
 (u'Foul Bunt', 24339),
 (u'Swinging Strike', 575348),
 (u'Swinging Strike (Blocked)', 41393),
 (u'Foul Tip', 47482),
 (u'Automatic Strike', 1),
 (u'Foul Pitchout', 1),
 (u'Foul', 1105452),
 (u'Ball', 2306759),
 (u'Strike', 48),
 (u'Swinging Pitchout', 21),
 (u'Intent Ball', 37744),
 (u'Called Strike', 1175482),
 (u'Unknown Strike', 83),
 (u'Pitchout', 4459),
 (u'Ball In Dirt', 123970),
 (u'Missed Bunt', 4639)]

We need to find all the pitches that rely solely on the ump's call: no fouls, intentional walks, swinging strikes, etc.  From the counts above we just want Called Strike and Ball.  Then we can check their positions relative to the strike zone for the batter and see how accurate we've been.  

In [11]:
relevant_pitchesRDD = pitchRDD.filter(lambda x: x[1][1] in ["Ball", "Called Strike"])

Count how many pitches that leaves us with:

In [12]:
relevant_pitchesRDD.count()

3482241

Define a function for checking whether the pitch is a strike and then comparing it to the ump's call.

In [28]:
errorCount = sc.accumulator(0)

In [29]:
def correctCall(sz_top, sz_bot, px, pz, call):
    if "null" in [sz_top, sz_bot, px, pz]:
        errorCount.add(1)
    else:
        if pz > sz_top or pz < sz_bot or px < -8.5 or px > 8.5:
            yield call == "Ball"
        else:
            yield call == "Called Strike"

In [30]:
accuracy_count = relevant_pitchesRDD.flatMap(lambda x: correctCall(x[1][8], x[1][9], x[1][12], x[1][13], x[1][1]))\
.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y).collect()

We ignored 2.3k pitches because of incomplete data.

In [31]:
print errorCount

233641


In [32]:
print accuracy_count

[(False, 1099141), (True, 2149459)]


Calculate the accuracy as: $$\frac{goodCalls}{goodCalls + badCalls}$$

Calculate the accuracy as: $$\frac{\Sigma\space correctCalls}{\Sigma\space correctCalls + \Sigma\space incorrectCalls}$$

In [36]:
print 1.*accuracy_count[1][1]/(accuracy_count[0][1] + accuracy_count[1][1])

0.661657021486


It doesn't look good.  When it all relies on him (no fouls, swinging strikes, intentional walks, etc), it looks like the ump only gets the right call 66% of the time.  There must be more to this since I can't believe the accuracy is actually that now.

For now, we can take a more fine-grained look at the calls.  Accuracy just gives an idea of how many calls are correct.  Next we'll look at precision and recall.

Precision compares the correctly called strikes to the incorrectly called strikes

In [44]:
errorCount = sc.accumulator(0)

In [45]:
def precisionStrikes(sz_top, sz_bot, px, pz, call):
    if "null" in [sz_top, sz_bot, px, pz]:
        errorCount.add(1)
    else:
        #if it's a called strike:
        if call == "Called Strike":
        #if it's a ball:
            if pz > sz_top or pz < sz_bot or px < -8.5 or px > 8.5:
                yield "Actual Ball"
            else:
                yield "Actual Strike"
            

In [46]:
precision_numbers = relevant_pitchesRDD.flatMap(lambda x: precisionStrikes(x[1][8], x[1][9], x[1][12], x[1][13], x[1][1]))\
.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y).collect()

In [47]:
print precision_numbers

[('Actual Ball', 1099141)]


# OKAY THIS IS AN ISSUE

I simply cannot believe that *literally* every called strike was in error.  Something's wrong with my interpretation of the data.  We'll come back to this next week.

Recall compares the called strikes to the actual strikes:  $$\frac{\Sigma\space correctStrikes}{\Sigma\space correctStrikes + \Sigma\space incorrectBalls}$$

In [ ]:
errorCount = sc.accumulator(0)
def recallStrikes(sz_top, sz_bot, px, pz, call):
    if "null" in [sz_top, sz_bot, px, pz]:
        errorCount.add(1)
    else:
        #if it's not a ball:
        if not (pz > sz_top or pz < sz_bot or px < -8.5 or px > 8.5):
            if call == "Called Strike":
                yield "Called Strike"
            else:
                yield "Missed Strike"
        else:
            yield call == "Called Strike"

### Thoughts so far

So this could all be done much more simply and much faster with a database, but next week (taking the lessons learned from sqoop and data acquisition) maybe we can do some actual data science on it to figure out what the deal is.